In [1]:
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt

from time import time
from functools import partial

import sys
sys.path.append('../../')

from Environment import SpringMass
from Subroutines import ModelEstV2,ModelEstV3,ModelEstV4,ApproxDAP,max_norm,find_stable_radius,SafeTransit,spectral_radius
from time import time
from Controllers import SafeDAP
from scipy.linalg import sqrtm
import pickle as pkl
%load_ext autoreload
%autoreload 2

In [2]:
class TimeStepsReached(Exception):
    pass
def evaluate(space_dim,dt,K_m_dt,l_m_dt):
    O = np.zeros((space_dim,space_dim))
    I = np.eye(space_dim)

    A = np.vstack([np.hstack([O,dt*I]),
                   np.hstack([-K_m_dt*I,-l_m_dt*I])]) + np.eye(2*space_dim)

    return  np.max(np.abs(np.linalg.eigvals(A)))

# Set up the system

## Find a highly stable spring-mass system

In [3]:
H_benchmark = 100
eps = 0.1

rho_target = eps**(1/H_benchmark)


In [4]:
rho_target

0.9772372209558107

Spectral radius of A: 0.9751294242564014 0.08140301073281651 0.4993663605670724
Step 0
Step 100
rho(A_pow[-1]) 0.08057979069717361 Sum(A_pow) 127.11901887229155 dx/w_max 11.165511157357344


In [9]:
w_max = 0.5


dt = 0.1
space_dim = 1
m = 1

candidate_param = np.random.rand(1000,2) * 0.05
istar = np.argmin([evaluate(space_dim,dt,K_m_dt,l_m_dt) for (K_m_dt,l_m_dt) in candidate_param])

K_star, l_star = candidate_param[istar]


K_star,l_star

K = K_star * m/dt

l = l_star*m/dt

print('Spectral radius of A:',evaluate(space_dim,dt,K_star,l_star),K,l)

x_0 = np.array([0,0]).reshape(-1,1)
time_steps = 200

env = SpringMass(x_0 = x_0,m=m,K_stab = K,l=l,w_adv_fraction=0,w_max=w_max)


x_rand_disturbance = []
# main loop
for _ in range(time_steps):
    x = env.state()
    u = 0 
        
    env.step(u)
    
    
    x_rand_disturbance.append(x)
    
    if _%100 == 0:
        print('Step',_)


# plt.plot(np.array(x_hist)[:,0],label='SafeDAP')
plt.figure(dpi=200)
plt.plot(np.array(x_rand_disturbance)[:,0],label=r'$w_{rand}$')
plt.xlabel('t')
plt.ylabel('x')
plt.legend()


A = env.A
B = env.B

A_pow = [np.eye(2)]
for i in range(H_benchmark):
    A_pow.append(A_pow[-1].dot(A))
            
print('rho(A_pow[-1])',spectral_radius(A_pow[-1]),'Sum(A_pow)',np.sum([np.linalg.norm(A,1) for A in A_pow]),'dx/w_max',np.max(x_rand_disturbance)/w_max)


# Experiments

## Experiment parameters

In [10]:
### Find a region(B(A,eps_init)) around A and ensure it contains only stable matrices.
eps_init = find_stable_radius(A)

est = ModelEstV2(A,B,eps_init) # The model estimator.
# est = ModelEstV3() # The model estimator.
# est = ModelEstV4() # The model estimator.

H = 20# Look back length



# Constants
x_dim = A.shape[-1]
u_dim = B.shape[-1]

I_x = np.eye(x_dim)  
I_u = np.eye(u_dim)  


w_cov = np.eye(x_dim) * env.w_max * 1/12 # Assume uniform distribution.

# Optimaization Problem Parameters

Q = np.array([[1,0],
                [0,1]])*1
R = np.eye(u_dim)*1

D_x = np.vstack([I_x,-I_x]) # Constraints4
d_x = np.array([6.0,20.0,6.0,20.0])

D_u = np.vstack([I_u,-I_u])
d_u = np.array([20.0,20.0])


M0 = np.zeros((H,u_dim,x_dim))

e_x = 0 # Usually e_x is not zero 
e_u = 0 # For safe DAP with known B, e_u = 0

refit_per_step = 50
pre_run_steps = 2
TD_steps  = 30 # The least number of steps taken in Phase 1.

# eta_bars = [0.1,0.5,1.0,1.5,2.0]
eta_bars = [0.1]

# eta_bars = [0.1]

params = {'Q':Q,'R':R,'D_x':D_x,'d_x':d_x,'D_u':D_u,'d_u':d_u,'A':A,'B':B,'H':H,'H_benchmark':H_benchmark,'eta_bars':eta_bars,'w_max':w_max}
with open('./data/Parameters.pkl','wb') as f:
    pkl.dump(params,f)

In [11]:

def PlainSim(timesteps,n_trials):
    
    def main_loop():
        env = SpringMass(x_0 = x_0,m=m,K_stab = K,l=l,w_adv_fraction=0,w_max = w_max)

        x_no_control_hist = []
        # main loop
        for _ in range(timesteps):
            x = env.state()
            u = 0 
            env.step(u)

            x_no_control_hist.append(x)

            if _%100 == 0:
                print('Step',_)

        data = {}
        data['x'] = x_no_control_hist
        return data
    
    trial_data = []
    _ = 0 
    while _ < n_trials: 
        t = time()
        print('Trial {}'.format(_))
        trial_data.append(main_loop())
        _+=1

        print('Time for trial:',time()-t)
    return trial_data

def BenchmarkSim(timesteps,n_trials,unconstrained=False):

    def main_loop(M):
        # Reset the environment
        env = SpringMass(x_0 = x_0,m=m,K_stab = K,l=l,w_adv_fraction=0,w_max = w_max)

        x_true_hist = []
        u_true_hist = []
        w_true_hist = [np.zeros((x_dim,1)) for _ in range(10*H_benchmark)]

        # main loop 
        for _ in range(timesteps):
            x = env.state()
            x_true_hist.append(x)
            if _<pre_run_steps:
                u = (np.random.rand()-0.5)*2*0
            else:
                u = ApproxDAP(M,w_true_hist,0)

            env.step(u)
            u_true_hist.append(u)

            w_true_hist.append(env.state()-A.dot(x_true_hist[-1])-B.dot(u_true_hist[-1]))

            if _%100 == 0:
                print('Step',_)

        # Prepare data output
        data = {}
        data['x'] = x_true_hist
        data['u'] = u_true_hist
        data['w'] = w_true_hist
        
#         print(data['w'])
        return data



    safeDapSolver = SafeDAP(Q,R,D_x,d_x,D_u,d_u,w_max,w_cov)

    if unconstrained:
        M,Phi = safeDapSolver.solve(A,B,H_benchmark,e_x=e_x,e_u=e_u,unconstrained=True)
    else:
        M,Phi = safeDapSolver.solve(A,B,H_benchmark,e_x=e_x,e_u=e_u)
    
#     print('M',M)

    trial_data = []
    _ = 0 
    while _ < n_trials: 

        print('Trial {}'.format(_))
        trial_data.append(main_loop(M))
        _+=1
        
    return trial_data    

In [12]:

n_trials = 3
timesteps = 400

alg_dict = {
    'Plain':PlainSim,
            'UnconstrainedDAP':partial(BenchmarkSim,unconstrained=True),
            'ConstrainedDAP':partial(BenchmarkSim,unconstrained=False)
           }


for key, alg in alg_dict.items():
    print("{} starts.".format(key))
    
    t = time()
      

    trial_data = alg(timesteps,n_trials)
    with open('./data/{}.pkl'.format(key),'wb') as f:
        pkl.dump(trial_data,f)
    
    print('Total Time for {}:'.format(key),time()-t)


Plain starts.
Trial 0
Step 0
Step 100
Step 200
Step 300
Time for trial: 0.0298159122467041
Trial 1
Step 0
Step 100
Step 200
Step 300
Time for trial: 0.013447046279907227
Trial 2
Step 0
Step 100
Step 200
Step 300
Time for trial: 0.013153076171875
Total Time for Plain: 0.06062173843383789
UnconstrainedDAP starts.
Trial 0
Step 0
Step 100
Step 200
Step 300
Trial 1
Step 0
Step 100
Step 200
Step 300
Trial 2
Step 0
Step 100
Step 200
Step 300
Total Time for UnconstrainedDAP: 3.7616124153137207
ConstrainedDAP starts.
Trial 0
Step 0
Step 100
Step 200
Step 300
Trial 1
Step 0
Step 100
Step 200
Step 300
Trial 2
Step 0
Step 100
Step 200
Step 300
Total Time for ConstrainedDAP: 16.401451587677002


In [13]:
for eta_bar in eta_bars:

    def SafeAdaptiveSim(timesteps,unconstrained=False):
        # Data containers

        x_hist = []
        u_hist = []
        w_hat_hist = [np.zeros((x_dim,1)) for _ in range(10*H)]

        theta_hist = []
        refit_time_step = []
        phase1_endpoints =[]
        phase2_endpoints=[]
        M_hist = [M0]
        r_hist = []

        n_episode = 0

        def phase(A_hat,B_hat,old_eta,new_eta):
            # Local variables: A_hat,B_hat,old,new.
            # Global variables: theta_hist,M_hist,e_x,e_u,H,env,w_max, w_hat_hist
            # Start of the phase
            # Solve for new controller M.
            if unconstrained:
                M,Phi = safeDapSolver.solve(A_hat,B_hat,H,e_x=e_x,e_u=e_u,unconstrained=True)
            else:
                M,Phi = safeDapSolver.solve(A_hat,B_hat,H,e_x=e_x,e_u=e_u)

            if n_episode==1:
                 old = {'M':np.array(M),'theta':(np.array(A_hat),np.array(B_hat)),'eta':old_eta,'r':r_hist[-1]}
            else:
                old = {'M':np.array(M_hist[-1]),'theta':theta_hist[-2],'eta':old_eta,'r':r_hist[-2]}
            new = {'M':np.array(M),'theta':theta_hist[-1],'eta':new_eta,'r':r_hist[-1]}

            old['e_x']=new['e_x']=e_x
            old['e_u']=new['e_u']=e_u

            M_hist.append(np.array(M))

            # Compute mid.
            mid = safeDapSolver.mid(old,new,H)

            transit = SafeTransit(old,new,mid,H)

            for i in range(refit_per_step):
                x = env.state()
                u = transit.get_u(w_hat_hist)
                env.step(u)

                Ah,Bh = transit.get_theta()
                # Calculate the estimated disturbance, and project it onto the bounded disturbance set. 
                w_hat = env.state()-Ah.dot(x)-Bh.dot(u)
                w_hat[w_hat>w_max] = w_max
                w_hat[w_hat<-w_max] = -w_max 
                w_hat_hist.append(np.array(w_hat)) 

                u_hist.append(u)
                x_hist.append(x)

                if len(x_hist)>=timesteps:
                    raise TimeStepsReached # Use the exception mechanism to stop the main loop once the total timestep is reached.

                # Updating M in the transit object.
                transit.step()
            # End of the phase


        env = SpringMass(x_0 = x_0,m=m,K_stab = K,l=l,w_adv_fraction=0,w_max=w_max)

        safeDapSolver = SafeDAP(Q,R,D_x,d_x,D_u,d_u,w_max,w_cov)

        try:
        # main loop
            while True:

                if n_episode==0:
                    for i in range(pre_run_steps):
                        x = env.state()
                        x_hist.append(x)
                        u = (np.random.rand()-0.5)*2*eta_bar
                        env.step(u)
                        u_hist.append(u)
                    A_hat,B_hat,r = est.est(np.array(x_hist).reshape(-1,x_dim),np.array(u_hist[:-1]).reshape(-1,u_dim))
#                     print(A_hat,B_hat)
                    theta_hist.append((np.array(A_hat),np.array(B_hat)))
                    r_hist.append(float(r))
                    refit_time_step.append(len(x_hist))
    #                 print(A_hat,B_hat)

                    # Fit the system parameters for the first time.
                else:

                    # Phase 1: exploration exploitation 

#                     print('phase 1')
                    phase(A_hat,B_hat,0,eta_bar)

                    phase1_endpoints.append(len(x_hist))

                    # Refit the system model
                    A_hat,B_hat,r = est.est(np.array(x_hist).reshape(-1,x_dim),np.array(u_hist[:-1]).reshape(-1,u_dim))
#                     print(A_hat,B_hat)
                    theta_hist.append((np.array(A_hat),np.array(B_hat)))
                    r_hist.append(float(r))
                    refit_time_step.append(len(x_hist))

                    # Phase 2: pure exploitation 
#                     print('phase 2')
                    phase(A_hat,B_hat,eta_bar,0)

                    phase2_endpoints.append(len(x_hist))


                    print('Episode',n_episode)

                n_episode+=1

        except TimeStepsReached:
            print('Time Steps Reached')
            # Prepare data output


        # Prepare data output
        data = {}
        data['x'] = x_hist
        data['u'] = u_hist
        data['w'] = w_hat_hist
        data['theta'] = theta_hist
        return data
    
    alg_dict = {'SafeLearning':partial(SafeAdaptiveSim,unconstrained=False),
               'UnconstrainedLearning':partial(SafeAdaptiveSim,unconstrained=True)
               }


    n_trials = 3
    time_steps = 400
    for key, alg in alg_dict.items():
        print("{} starts.".format(key))
        trial_data = []
        _ = 0 
        while _ < n_trials: 
            t = time()
            try:
                print('Trial {}'.format(_))
                trial_data.append(alg(time_steps))
                _+=1
            except AttributeError:
                print('Infeasibility Encountered. Restarting the loop.')
            except cp.SolverError:
                print('Solver Error Encountered. Restarting the loop.')
            
            print('Time for trial:',time()-t)
            
        with open('./data/{}_eta_{}.pkl'.format(key,eta_bar),'wb') as f:
            pkl.dump(trial_data,f)

from IPython.display import Audio

sound_file = '../../sound/Bike-bell-sound.wav'
Audio(sound_file,autoplay=True)

SafeLearning starts.
Trial 0
Episode 1
Episode 2
Episode 3
Time Steps Reached
Time for trial: 15.875370979309082
Trial 1
Episode 1
Episode 2
Episode 3
Time Steps Reached
Time for trial: 15.462303876876831
Trial 2
Episode 1
Episode 2


/home/tianpeng/miniconda3/envs/simulation/lib/python3.9/site-packages/cvxpy/problems/problem.py:1294: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Episode 3
Time Steps Reached
Time for trial: 15.434927940368652
UnconstrainedLearning starts.
Trial 0
Episode 1
Episode 2
Episode 3
Time Steps Reached
Time for trial: 12.061899900436401
Trial 1
Episode 1
Episode 2
Episode 3
Time Steps Reached
Time for trial: 12.001426458358765
Trial 2
Episode 1
Episode 2
Episode 3
Time Steps Reached
Time for trial: 12.044068098068237
